In [174]:
import numpy as np
import cvxpy as cp

class Car:
    
    def __init__(self, start_pos=None, start_angle=None):
        if start_pos is None:
            self.player_pos = np.array([[100.], [100.]])
        else:
            self.player_pos = start_pos
        self.player_speed = 0.
        self.player_accelaration = 0.
        if start_angle is None:
            self.player_direction = np.array([[1/np.sqrt(2)], [1/np.sqrt(2)]])
        else:
            self.player_direction = start_angle
        self.wheel_direction = self.player_direction
        self.car_size = 40.
        
    def update(self):
        front_position = self.player_pos + self.car_size * self.player_direction
        self.player_pos += self.player_speed * self.player_direction
        new_front_position = front_position + self.player_speed * self.wheel_direction
        new_direction = new_front_position - self.player_pos
        self.player_direction = new_direction / np.linalg.norm(new_direction)
        
    def update_controls(self, update):
        self.player_accelaration = update[1]
        current_angle = np.arctan(self.player_direction[1]/self.player_direction[0])
        if self.player_direction[0] < 0:
            current_angle += np.pi
        new_angle = current_angle + update[0]
        self.wheel_direction = np.array([[np.cos(new_angle)[0]], [np.sin(new_angle)[0]]])
        self.player_speed += self.player_accelaration
    
    def step(self, angle, accelaration):
        unit_vec = np.array([np.cos(angle), np.sin(angle)])
        update = np.array([angle, accelaration])
        self.update()
        self.update_controls(update)

In [175]:
def solve_linearized_problem(cars, index, prev_acc, dest):
    car = cars[index]
    L = car.car_size
    xt = car.player_pos
    yt = car.player_pos + car.player_direction * car.car_size
    dt = car.player_direction
    vt = car.player_speed
    delta_at = cp.Variable()
    wt = cp.Variable()
    wtp1 = cp.Variable()
    xtp1 = xt + dt * vt
    ytp1 = cp.Variable((2,1))
    ytp2 = cp.Variable((2,1))
    atm1 = prev_acc
    Wt1 = cp.Variable((2,2))
    constraints = [Wt1[0,0] == 0, Wt1[0,1] == wt, Wt1[1,0] == -wt, Wt1[1,1] == 0]
    Wt2 = cp.Variable((2,2))
    constraints += [Wt2[0,0] == 0, Wt2[0,1] == wtp1, Wt2[1,0] == -wtp1, Wt2[1,1] == 0]
    Da = cp.Variable((2,2))
    constraints += [Da[0,0] == delta_at, Da[0,1] == 0, Da[1,0] == 0, Da[1,1] == delta_at]
    On = np.array([[1, 0], [0, 1]])
    constraints += [ytp2 == ytp1 + ((On + Wt2) @ yt + (On + Wt1 + Wt2) @ dt * L * vt - (On + Wt2) @ xtp1) * L * (atm1 + vt)
                            + ((Da) @ yt + (Da) @ dt * L * vt - (Da) @ xtp1) * L]
    constraints += [ytp1 == yt + ((On + Wt1) @ dt * L * vt)]
    constraints += [cp.abs(delta_at) <= 5, cp.abs(wt) <= 0.5, cp.abs(wtp1) <= 0.5]
    objective = cp.sum_squares(dest - ytp2)**2
    problem = cp.Problem(cp.Minimize(objective), constraints)
    problem.solve()
    return wt.value, delta_at.value

In [199]:
def solve_linearized_problem_2(cars, index, dest):
    car = cars[index]
    L = car.car_size
    xt = car.player_pos
    yt = car.player_pos + car.player_direction * car.car_size
    dt = car.player_direction
    vt = car.player_speed
    at = cp.Variable()
    wt = car.wheel_direction
    wtp1 = cp.Variable()
    xtp1 = xt + dt * vt
    ytp1_ = yt + vt * wt
    ytp1 = xtp1 + L * (ytp1_ - xtp1)/np.linalg.norm(ytp1_ - xtp1)
    ytp2 = cp.Variable((2,1))
    Wt2 = cp.Variable((2,2))
    
    constraints = [Wt2[0,0] == 0, Wt2[0,1] == wtp1, Wt2[1,0] == -wtp1, Wt2[1,1] == 0]
    On = np.array([[1, 0], [0, 1]])
    
    constraints += [ytp2 == ytp1 + (On + Wt2) @ (ytp1 - xtp1)/L * vt + (at * On) @ (ytp1 - xtp1)/L]
    constraints += [cp.abs(at) <= 0.5, cp.abs(wtp1) <= 0.25]
    objective = cp.sum_squares(dest - ytp2)
    problem = cp.Problem(cp.Minimize(objective), constraints)
    problem.solve()
    return -wtp1.value, at.value

In [200]:
import pygame

window_size = 500
frame_rate = 30
car_colours = [np.array([255., 0., 0.]), np.array([0., 255., 0.])]

car_starts = [np.array([[40.], [180.]]), np.array([[180.], [40.]])]
start_angles = [np.array([[1], [0]]), np.array([[0], [1]])]
car_destinations = [np.array([[420.], [100.]]), np.array([[300.], [420.]])]
cars = [Car(car_starts[0], start_angles[0]), Car(car_starts[1], start_angles[1])]

def main():
    accelaration = 0
    pygame.init()
    win = pygame.display.set_mode((window_size, window_size))
    pygame.display.set_caption("Cars")
    font = pygame.font.SysFont('freesansbold.ttf', 80)
    small_font = pygame.font.SysFont('freesansbold.ttf', 20)
    clock = pygame.time.Clock()
    
    def draw_board(cars, destinations):
        for i in range(len(cars)):
            car = cars[i]
            rear_position = car.player_pos
            centre_position = car.player_pos + car.player_direction * car.car_size / 2
            end_position = car.player_pos + car.player_direction * car.car_size
            unrotated_tl = np.array([centre_position[0] - car.car_size//2, centre_position[1]- car.car_size//2])
            rect = pygame.Rect(np.array([0., 0.]), (1, 1))
            angle = np.arctan(car.player_direction[0]/car.player_direction[1])
            if car.player_direction[1] < 0:
                angle += np.pi
            #surf = pygame.Surface((int(car.car_size), int(car.car_size)))
            ##surf.fill(car_colours[i])
            #pygame.draw.rect(surf, (0, 0, 0), rect)
            #new_surf = pygame.transform.rotate(surf, angle*180/np.pi)
            #win.blit(new_surf, centre_position.flatten())
            pygame.draw.circle(win, car_colours[i] + np.array([0, 0, 100]), (car_destinations[i][0,0], car_destinations[i][1,0]), 10)
            pygame.draw.circle(win, car_colours[i], (end_position[0,0], end_position[1,0]), 30)
            pygame.draw.circle(win, np.array([0., 0., 255.]), (centre_position[0,0], centre_position[1,0]), 30)
            pygame.draw.circle(win, car_colours[i], (car.player_pos[0,0], car.player_pos[1,0]), 30)
            
            
            
    run = True
    while run:
        clock.tick(frame_rate)
        win.fill((0, 0, 0))
        draw_board(cars, car_destinations)
        pygame.display.update()
        pygame.event.pump()
        keys = pygame.key.get_pressed()
        
        angle = 0; accelaration = 0
        if keys[pygame.K_w]:
            accelaration = 1
        if keys[pygame.K_s]:
            accelaration = -1
        if keys[pygame.K_a]:
            angle = -0.5
        if keys[pygame.K_d]:
            angle = 0.5
        if keys[pygame.K_b]:
            if cars[0].player_speed > 1:
                accelaration = -3
            elif cars[0].player_speed < -1:
                accelaration = 5
            elif cars[0].player_speed > -1 and cars[0].player_speed < 1:
                accelaration = 0
                cars[0].player_speed = 0
                
        angle1, acc1 = solve_linearized_problem_2(cars, 0, car_destinations[0])
        angle2, acc2 = solve_linearized_problem_2(cars, 1, car_destinations[1])
        
        cars[0].step(angle1, acc1)
        cars[1].step(angle2, acc2)
        cars[0].player_speed = 0.9 * cars[0].player_speed
        cars[1].player_speed = 0.9 * cars[1].player_speed
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False

main()

<ipython-input-200-1fa39bafaa91>:29: RuntimeWarning: divide by zero encountered in true_divide
  angle = np.arctan(car.player_direction[0]/car.player_direction[1])


In [ ]:
pygame.Rect(np.array([[0.], [0.]]).flatten(), (10, 10))

In [27]:
game.step(0.5, 1.)

In [28]:
game.player_pos

array([[101.],
       [100.]])

In [29]:
game.step(0.5, 1.)

In [30]:
game.player_pos

array([[102.99215284],
       [100.17699458]])

In [31]:
game.step(0.5, 1.)

In [32]:
game.player_pos

array([[105.90030033],
       [100.9136615 ]])

In [49]:
X = np.array([[1], [2]])

In [50]:
X.flatten()

array([1, 2])

In [51]:
X

array([[1],
       [2]])

In [66]:
correct_P = np.array([[3/5, 3/10, 0, 1/10],
                      [0, 3/5, 3/10, 1/10],
                      [2/5, 0, 2/5, 2/10],
                      [0, 0, 0, 1]])

correct_P @ np.array([17/2, 8, 22/3, 0]) + 1

array([8.5       , 8.        , 7.33333333, 1.        ])

In [76]:
bad_P = np.array([[2/3, 7/30, 0],
                      [0, 2/3, 7/30],
                      [3/10, 0, 1/2]])

np.linalg.inv(bad_P - np.eye(3)) @ np.array([-1, -1, -1])

In [82]:
np.linalg.inv(bad_P - np.eye(3)) @ np.array([-1, -1, -1])

array([8.61189802, 8.01699717, 7.16713881])

In [87]:
7 * 283/353 + 3 * 353/353

8.611898016997166

In [85]:
(0.4 * 0.7 + 0.3 * 1.3)/((0.4)**2*0.7 + 0.4 * 0.7**2)

2.175324675324675

In [88]:
3040/353

8.611898016997166

In [89]:
5472/900 * 9000/6354

8.611898016997166

In [90]:
3040/353

8.611898016997166

In [91]:
77*21/130

12.438461538461539

In [109]:
6040/1353

4.464153732446415

In [95]:
152*20/353

8.611898016997166

In [96]:
7/10 * (7 * 3.8/10 + 3) * (1000/(1000 - 7*42))

5.611898016997166

In [97]:
304*30/1059

8.611898016997166

In [98]:
3 + 7/10 * 500/353 * 849/150

8.611898016997166

In [99]:
608/706

0.8611898016997167

In [101]:
3040/451 + 5168/4510 + 4

11.886474501108648

In [102]:
1/0.4 + 3/4*(5 + 3/4*(1/0.3))

8.125

In [103]:
10 * (30 * 30 * 3 + 63 + 21 * 42)/(30 * 300 - 126*21)

5.736543909348442

In [104]:
325/46

7.065217391304348

In [105]:
145/8

18.125

In [106]:
4500/1059 * (304/150)

8.611898016997168

In [107]:
8100/753

10.756972111553784

In [108]:
98*15/259

5.675675675675675

In [110]:
173054/70600

2.4511898016997167

In [111]:
6040/1353

4.464153732446415

In [112]:
3040/353

8.611898016997166